# 半监督学习
***半监督学习(Semi-Supervised Learning，SSL)*** 是模式识别和机器学习领域研究的重点问题，是监督学习与无监督学习相结合的一种学习方法。半监督学习使用大量的未标记数据，以及同时使用标记数据，来进行模式识别工作。当使用半监督学习时，将会要求尽量少的人员来从事工作，利用少量的已标注数据进行指导并预测未标记数据的标记，并合并到标记数据集中去；同时，又能够带来比较高的准确性，

### 算法思路：
- 生成模型：先计算样本特征的总体的联合分布，将所有有标注的样本计算出一个分布，然后把没有标注的样本放入这个分布中，看根据这个分布它该如何被标注，这个过程可能是迭代的
- 物以类聚：将有标注和没有标注的样本进行相似的比较，相似度高的，就将无标注样本按照临近的有标注样本进行标注，类似迭代过程。

#### 下面着重关注第二种，涉及的算法是标签传播算法
### 标签传播算法
标签传播算法（Label Propagation Algorithm）是基于图的半监督学习方法，基本思路是从已标记的节点的标签信息来预测未标记的节点的标签信息，利用样本间的关系，建立完全图模型。

每个节点标签按相似度传播给相邻节点，在节点传播的每一步，每个节点根据相邻节点的标签来更新自己的标签，与该节点相似度越大，其相邻节点对其标注的影响权值越大，相似节点的标签越趋于一致，其标签就越容易传播。在标签传播过程中，保持已标记的数据的标签不变，使其将标签传给未标注的数据。最终当迭代结束时，相似节点的概率分布趋于相似，可以划分到一类中。

标签传播表示半监督图推理算法的几个变种，且它有如下2个特性：
- 可用于分类和回归任务
- 将数据投射到另一维空间的内核方法(Kernel methods )

值得注意的是，sklearn.semi_supervised中的半监督估计器能够利用这些额外的未标签数据来更好地捕捉到底层数据分布的形状(the shape of the underlying data distribution)，并对新的样本进行更好的泛化。当我们有极少量的有标签数据和大量的无标签数据时，半监督算法可以取得较好的表现。

scikit-learn提供了两种标签传播模型 --- LabelPropagation和LabelSpreading，这两种模型的工作原理是在输入数据集的所有项上构建一个相似度图(similarity graph)。
LabelPropagation和LabelSpreading有相同点，也存在明显的差异：
- LabelPropagation和LabelSpreading对图的相似度矩阵( similarity matrix)的修改，以及对标签分布的箝位效应( the clamping effect )。箝位允许算法在一定程度上改变真的标签数据的权重。LabelPropagation算法对输入标签进行硬箝位(hard clamping)，也就是a = 0，LabelPropagation算法对输入标签进行硬箝位。这个箝位系数可以放宽，可以说是a = 0.2，这意味着我们将始终保留80%的原始标签分布，但算法得到的置信度在20%以内。

- LabelPropagation使用从数据中构建的原始相似度矩阵（the raw similarity matrix），不做任何修改。相比之下，***LabelSpreading将具有正则化特性的损失函数最小化，因此它通常对噪声更稳健（划重点）。*** 该算法对原始图的修改版本进行迭代，并通过计算归一化的图Laplacian矩阵对边缘权重进行归一化。这个过程也被用于频谱聚类（Spectral clustering）中。

- LabelSpreading类似于基本的标签传播算法（The basic Label Propagation algorithm），但使用了基于归一化的graph Laplacian和soft clamping 的亲和矩阵（affinity matrix）在标签间进行传播。

标签传播模型有两种内置的内核方法。内核的选择对算法的可扩展性和性能都有影响。有以下2种方法可供选择。
- rbf，距离离的越近越接近于1，距离离的越远越接近于0，由关键字gamma指定。
- knn，找一个无标注的数据，然后取附近k个有标注的数据，无标注数据附近哪种标注的数据最多就取哪一个（以未标注的数据为圆心做knn，在指定范围内找到了有标注的数据，然后对未标注的数据进行打标，然后进行打标传播，直到未标注的数据全都标注以后，算法结束），由关键字n_neighbors指定。

rbf内核将产生一个完全连接的图，在内存中用一个密集矩阵表示。这个矩阵可能非常大，再加上算法每次迭代都要进行一次完整的矩阵乘法计算，会导致运行时间过长。另一方面，KNN 内核将产生一个更有利于内存的稀疏矩阵，可以大大减少运行时间。
由于LabelSpreading有较好的抗噪性，笔者将在下面的实例中使用该方法对含有少量标注数据和大量无标注的数据的样本集进行基于标签传播（Label Propagation）的无监督学习。
为了方便，用于演示的数据来自sklearn自带的20newsgroups数据集，目前测试下来，这个半监督方法在长文本分类项目上效果奇好；如果是短文本的话，提取特征得用到当下最先进的transformer系预训练模型了。

## 利用sklearn中的LabelSpreading进行小样本学习

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.datasets import fetch_20newsgroups
from sklearn import datasets
from sklearn.semi_supervised import LabelSpreading
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# 作为测试，只需要其中的5类即可，且训练集和测试集都用到，即参数“subset”取“all”。
categories=[
    'rec.autos',
    'talk.politics.guns',
    'talk.politics.mideast',
    'rec.sport.baseball',
    'comp.sys.mac.hardware',
    'soc.religion.christian'
]
newsgroup_train=fetch_20newsgroups(subset='all',categories=categories)

In [13]:
# 将数据集打乱，随机化操作。
# newsgroup_train['data'][0]
rng=np.random.RandomState(0)
indices=np.arange(len(newsgroup_train.target))
rng.shuffle(indices)

In [14]:
# 抽取文本数据的特征，用到的是tf-idf特征，并用到1gram和2gram，并去掉停用词，频率超高65%的特征词排除，且最大特征数为15000（词表中的最大词汇数）。
vectorizer=TfidfVectorizer(
    stop_words='english',
    max_df=0.65,
    ngram_range=(1,2),
    max_features=15000
)

fea_train=vectorizer.fit_transform(newsgroup_train.data)
y_train=newsgroup_train.target

抽取文本数据的特征，用到的是tf-idf特征，并用到1gram和2gram，并去掉停用词，频率超高65%的特征词排除，且最大特征数为15000（词表中的最大词汇数）。

当然，你可以通过改变max_iterations来标注更多的标签。标记更多的标签标签可以帮助我们了解这种主动学习技术的收敛速度。

注意：当用拟合方法训练模型时，为未标记的点和标记的数据一起分配一个标识符是很重要的，本实例中使用的标识符是整型值-1。

In [15]:
test_num=2000
X=fea_train[indices[:test_num]]
y=y_train[indices[:test_num]]
images=np.array(newsgroup_train.data)[indices[:test_num]]

n_total_samples=len(y)
n_labeled_points=300
max_iterations=20

unlabeled_indices=np.arange(n_total_samples)[n_labeled_points:]

In [16]:
# 检视下未标注数据的index，注意这是随机的。
unlabeled_indices

array([ 300,  301,  302, ..., 1997, 1998, 1999])

在下面的每次迭代中，程序都会基于信息熵来显示其中机器***最拿不准的*** TOP10文本数据，这些数字可能包含错误，也可能不包含错误，这些数据其实是我们在语料标注中最需要标注的，它们对分类的影响极为重要，有时我们也可以在这些不确定的预标注数据中找到错误的标注。在这里，这些不确定样例将会使用它们的真实标签（True labels），投入到下一轮次的模型训练中。

In [18]:

for i in range(max_iterations):
    if len(unlabeled_indices) == 0:
        print("没有待打标的候选标签项")
        break
    y_train = np.copy(y)
    y_train[unlabeled_indices] = -1

    lp_model = LabelSpreading(
                        gamma=0.25, 
                        kernel='knn',
                        alpha = 0.5,
                        n_neighbors =15,
                        max_iter=50,
                        n_jobs = -1
                        )
    lp_model.fit(X.toarray(), y_train)

    predicted_labels = lp_model.transduction_[unlabeled_indices]
    true_labels = y[unlabeled_indices]

    cm = confusion_matrix(true_labels, predicted_labels,
                          labels=lp_model.classes_)

    print("【迭代轮次】 %i %s" % (i, 70 * "_"))
    print("LabelSpreading model: %d 个已标记 & %d 个未标记 (%d 个总数)"
          % (n_labeled_points, n_total_samples - n_labeled_points,
             n_total_samples))

    print(classification_report(
        true_labels, 
            predicted_labels,
            target_names = [
                     'rec.autos',
                     'talk.politics.guns',
                     'talk.politics.mideast',
                     'rec.sport.baseball',
                     'comp.sys.mac.hardware',
                     'soc.religion.christian']
            ))

    print("【混淆矩阵】")
    print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        lp_model.label_distributions_.T)

    # select up to 10 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:10]

    # keep track of indices that we get labels for
    delete_indices = np.array([], dtype=int)


    print('【最不确定样本呈现】\n',images)
    for index, image_index in enumerate(uncertainty_index):
        image = images[image_index]


        if i < max_iterations:

            print('……………'*5)
            print("预测标签: {}\n真实标签: {}".format(
                newsgroup_train.target_names[lp_model.transduction_[image_index]], newsgroup_train.target_names[y[image_index]]))
            print('******************'*5)

        # labeling 10 points, remote from labeled set
        delete_index, = np.where(unlabeled_indices == image_index)
        delete_indices = np.concatenate((delete_indices, delete_index))

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)
    print('=========第 {} 轮结束~============'.format(i))

【迭代轮次】 0 ______________________________________________________________________
LabelSpreading model: 300 个已标记 & 1700 个未标记 (2000 个总数)
                        precision    recall  f1-score   support

             rec.autos       0.72      0.74      0.73       282
    talk.politics.guns       0.75      0.77      0.76       281
 talk.politics.mideast       0.78      0.73      0.75       281
    rec.sport.baseball       0.87      0.85      0.86       319
 comp.sys.mac.hardware       0.84      0.81      0.82       268
soc.religion.christian       0.83      0.91      0.87       269

              accuracy                           0.80      1700
             macro avg       0.80      0.80      0.80      1700
          weighted avg       0.80      0.80      0.80      1700

【混淆矩阵】
[[209  18  20  13   5  17]
 [ 30 216  13  10   7   5]
 [ 17  29 204   8  13  10]
 [ 11  14   7 270   9   8]
 [ 14   9  12   8 216   9]
 [  8   2   7   0   8 244]]
【最不确定样本呈现】

…………………………………………………………………
预测标签: comp.sys.

KeyboardInterrupt: 